In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State


import os
import base64
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps
from read_write import AnimalShelter



###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "1234"
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))
water_df = pd.DataFrame.from_records(shelter.read(
    {"breed" : { "$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"] },
     "sex_upon_outcome" : "Intact Female",
     "age_upon_outcome_in_weeks" : { "$lte" : 156, "$gte" : 26 } }
))
mountain_df = pd.DataFrame.from_records(shelter.read(
    {"breed" : { "$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"] },
     "sex_upon_outcome" : "Intact Male",
     "age_upon_outcome_in_weeks" : { "$lte" : 156, "$gte" : 26 } }
))
disaster_df = pd.DataFrame.from_records(shelter.read(
    {"breed" : { "$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"] },
     "sex_upon_outcome" : "Intact Male",
     "age_upon_outcome_in_weeks" : { "$lte" : 300, "$gte" : 20 } }
))




#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'GSLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    #Create logo and unique identifier
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Michael Jordan SNHU CS-340 Dashboard'))),
    html.Center(
        html.A(
            href='https://www.snhu.edu',
            children=[
                html.Img(
                    src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    width=120,
                    height=120
                )
            ]
        )
    ),
    #Create interactive components
    html.Div([
        dcc.RadioItems(
            id='rescue-types',
            options=[
                {'label' : 'Water Rescue', 'value' : 'WR'},
                {'label' : 'Mountain/Wilderness Rescue', 'value' : 'MWR'},
                {'label' : 'Disaster Rescue/Tracking', 'value' : 'DRT'},
                {'label' : 'Reset Filters', 'value' : 'RF'}
            ],
            labelStyle={'display' : 'inline-block'}
        )
    ]),    
    #Create data table
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        page_action="native",
        page_current=0,
        page_size=10,
        style_cell_conditional=[
            {
                'if': {'column_id': c},
                'textAlign': 'left'
            } for c in ['animal_type', 'age_upon_outcome', 'breed', 'color', 'name', 'outcome_subtype',
                       'outcome_type', 'sex_upon_outcome']
        ],
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(230, 230, 230)'
            }
        ]

    ),
    #Create map & graph
    html.Br(),
    html.Hr(),
    html.Div(className='row',
            style={'display' : 'flex'},
            children=[
                html.Div(
                    id='graph-id',
                    className='col s12 m6'
                ),
                html.Div(
                    id='map-id',
                    className='col s12 m6'
                )
            ]
        )
])

#############################################
# Interaction Between Components / Controller
#############################################

#Function to update the data table
@app.callback(
    Output('datatable-id', 'data'),
    [Input('rescue-types', 'value')]
)
def update_filter(rescue_type):
    dff = df
    if(rescue_type == 'WR'):
        dff = water_df
    elif(rescue_type == 'MWR'):
        dff = mountain_df
    elif(rescue_type == 'DRT'):
        dff = disaster_df
    else:
        dff = df
    return dff.to_dict('records')

#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None:
        viewData = df.to_dict('records')
    fig = px.pie(viewData, names='breed')
    return [
        dcc.Graph(            
            figure = fig
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    aniName = "No name available" if dff.iloc[0,9] is "" else dff.iloc[0,9]
    aniLocation = [dff.iloc[0, 13], dff.iloc[0, 14]]
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=aniLocation, zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=aniLocation, children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(aniName)
                ])
            ])
        ])
    ]

app